# **Compliance GPT with LangChain**

## **Setup**

In [1]:
# import nest_asyncio
# nest_asyncio.apply()

## **Config**

In [2]:
from utils.config import get_config

config = get_config()

In [3]:
USER_ID = 'xmriz'
CONVERSATION_ID = 'xmriz-2021-07-01-01'

## **Define Model**

In [4]:
from utils.models import ModelName, LLMModelName, EmbeddingModelName, get_model

model_name = ModelName.AZURE_OPENAI
llm_model, embed_model = get_model(model_name=model_name, config=config, llm_model_name=LLMModelName.GPT_35_TURBO, embedding_model_name=EmbeddingModelName.EMBEDDING_3_SMALL)

## **Load Vector Store Index**

In [5]:
from database.vector_store.vector_store import RedisIndexManager, PineconeIndexManager


redis_bi = RedisIndexManager(index_name='bi', embed_model=embed_model, config=config, db_id=0)
vector_store_bi = redis_bi.load_vector_index()

redis_ojk = RedisIndexManager(index_name='ojk', embed_model=embed_model, config=config, db_id=0)
vector_store_ojk = redis_ojk.load_vector_index()

redis_sikepo_ket = RedisIndexManager(index_name='sikepo-ketentuan-terkait', embed_model=embed_model, config=config, db_id=0)
vector_store_ket = redis_sikepo_ket.load_vector_index()

redis_sikepo_rek = RedisIndexManager(index_name='sikepo-rekam-jejak', embed_model=embed_model, config=config, db_id=0)
vector_store_rek = redis_sikepo_rek.load_vector_index()

# pinecone_ojk = PineconeIndexManager(index_name='ojk', embed_model=embed_model, config=config)
# vector_store_ojk = pinecone_ojk.load_vector_index()

# pinecone_bi = PineconeIndexManager(index_name='bi', embed_model=embed_model, config=config)
# vector_store_bi = pinecone_bi.load_vector_index()

# pinecone_ket = PineconeIndexManager(index_name='ketentuan-terkait', embed_model=embed_model, config=config)
# vector_store_ket = pinecone_ket.load_vector_index()

# pinecone_rek = PineconeIndexManager(index_name='rekam-jejak', embed_model=embed_model, config=config)
# vector_store_rek = pinecone_rek.load_vector_index()

c:\Users\acer\miniconda3\envs\chatbot-ocbc\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
from database.vector_store.neo4j_graph_store import Neo4jGraphStore

neo4j_sikepo = Neo4jGraphStore(config=config)
graph = neo4j_sikepo.get_graph()

## **Get Retriever**

In [7]:
from retriever.retriever_ojk.retriever_ojk import get_retriever_ojk
from retriever.retriever_bi.retriever_bi import get_retriever_bi
from retriever.retriever_sikepo.lotr_sikepo import lotr_sikepo

retriever_bi = get_retriever_bi(vector_store=vector_store_bi, top_n=6, top_k=16, llm_model=llm_model, embed_model=embed_model, config=config)
retriever_ojk = get_retriever_ojk(vector_store=vector_store_ojk, top_n=6, top_k=16, llm_model=llm_model, embed_model=embed_model, config=config)
retriever_sikepo_ket = lotr_sikepo(vector_store=vector_store_ket, llm_model=llm_model, embed_model=embed_model, config=config)
retriever_sikepo_rek = lotr_sikepo(vector_store=vector_store_rek, llm_model=llm_model, embed_model=embed_model, config=config)

In [8]:
retriever_sikepo_ket.invoke("What is QRIS?")

score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.
score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.


[Document(metadata={'id': 'doc:sikepo-ketentuan-terkait:e5ff77ccf94c4a6ca4ddca6394ea301f', 'Nomor Ketentuan': '21 Tahun 2023', 'Ketentuan': 'Pasal 11 ', 'Referensi': '-', 'Kodifikasi Ketentuan': 'Bank Umum Konvensional', 'Jenis Ketentuan': 'POJK', 'Judul Ketentuan': 'Layanan Digital Oleh Bank Umum', 'Tanggal Ketentuan': '19 Desember 2023', 'Tanggal Ketentuan Unix': '1702918800', 'Tahun Ketentuan': '2023', 'relevance_score': 0.98005307}, page_content='serta memiliki penjelasan tambahan sebagai berikut: Yang dimaksud dengan "produk Bank dasar" adalah produk bank dasar sesuai dengan Peraturan Otoritas Jasa Keuangan mengenai penyelenggaraan produk bank umum. Layanan Digital yang perizinan sepenuhnya diatur oleh otoritas sistem pembayaran di Indonesia antara lain Quick Response Code Indonesian Standard/QRIS dan dompet elektronik. Contoh: 1. Bank telah memiliki Layanan Digital berupa aplikasi mobile banking yang telah dilengkapi dengan produk tabungan dan sebelumnya telah memperoleh izin dar

## **Create Chain**

In [9]:
from database.chat_store import RedisChatStore, MongoDBChatStore

chat_store = RedisChatStore(k=3, config=config, db_id=1)
# chat_store = MongoDBChatStore(config=config, k=4)

In [10]:
from chain.rag_chain import create_chain_with_chat_history, create_chain
from retriever.retriever_sikepo.graph_cypher_retriever import graph_rag_chain


graph_chain = graph_rag_chain(llm_model, llm_model, graph=graph)

chain = create_chain(
    llm_model=llm_model,
    graph_chain=graph_chain,
    retriever_ojk=retriever_ojk,
    retriever_bi=retriever_sikepo_ket,
    retriever_sikepo_ketentuan=retriever_sikepo_ket,
    retriever_sikepo_rekam=retriever_sikepo_rek,
)

chain_history = create_chain_with_chat_history(
    final_chain=chain,
    chat_store=chat_store,
)

## **Invoke**

#### **Chat Store Monitor**

In [11]:
# to print chat history
print(chat_store.get_session_history(user_id=USER_ID, conversation_id=CONVERSATION_ID))

In [12]:
# to make the chat history empty
chat_store.clear_all()

#### **With Question, Answer, and Context**

In [14]:
from chain.rag_chain import get_response

response = get_response(
    chain=chain_history,
    question="Apa itu Dana Tanahud?",
    user_id=USER_ID,
    conversation_id=CONVERSATION_ID
)

response

{'rewrited question': 'Apa itu Dana Tanahud?',
 'answer': 'Dana Tanahud adalah kumpulan dana yang berasal dari kontribusi tanahud, hasil investasi dana tanahud, qardh dari Perusahaan kepada dana tanahud, dan/atau dana tanahud dari reasuradur, yang penggunaannya sesuai dengan perjanjian anuitas syariah untuk program pensiun atau perjanjian reasuransi syariah atas anuitas syariah untuk program pensiun.\n\nSource: [POJK tentang Penyelenggaraan Usaha Perusahaan Asuransi, Perusahaan Asuransi Syariah, Perusahaan Reasuransi, dan Perusahaan Reasuransi Syariah](https://www.ojk.go.id/id/regulasi/Documents/Pages/POJK-tentang--Penyelenggaraan-Usaha-Perusahaan-Asuransi,-Asuransi-Syariah,-Reasuransi,-dan-Reasuransi-Syariah/pojk%2069-2016.pdf#page=7)',
 'context': '[\n  {\n    "metadata": {\n      "id": "doc:ojk:35dce9bb283f49048c149b24356c12e0",\n      "title": "POJK tentang Penyelenggaraan Usaha Perusahaan Asuransi, Perusahaan Asuransi Syariah, Perusahaan Reasuransi, dan Perusahaan Reasuransi Syari

In [15]:
print(response['answer'])


Dana Tanahud adalah kumpulan dana yang berasal dari kontribusi tanahud, hasil investasi dana tanahud, qardh dari Perusahaan kepada dana tanahud, dan/atau dana tanahud dari reasuradur, yang penggunaannya sesuai dengan perjanjian anuitas syariah untuk program pensiun atau perjanjian reasuransi syariah atas anuitas syariah untuk program pensiun.

Source: [POJK tentang Penyelenggaraan Usaha Perusahaan Asuransi, Perusahaan Asuransi Syariah, Perusahaan Reasuransi, dan Perusahaan Reasuransi Syariah](https://www.ojk.go.id/id/regulasi/Documents/Pages/POJK-tentang--Penyelenggaraan-Usaha-Perusahaan-Asuransi,-Asuransi-Syariah,-Reasuransi,-dan-Reasuransi-Syariah/pojk%2069-2016.pdf#page=7)


In [16]:
print(response['context'])

[
  {
    "metadata": {
      "id": "doc:ojk:35dce9bb283f49048c149b24356c12e0",
      "title": "POJK tentang Penyelenggaraan Usaha Perusahaan Asuransi, Perusahaan Asuransi Syariah, Perusahaan Reasuransi, dan Perusahaan Reasuransi Syariah",
      "sector": "IKNB,  Syariah",
      "subsector": "IKNB Syariah,  Asuransi",
      "regulation_type": "Peraturan OJK",
      "regulation_number": "69/POJK.05/2016",
      "effective_date": "28 Desember 2016",
      "file_url": "https://www.ojk.go.id/id/regulasi/Documents/Pages/POJK-tentang--Penyelenggaraan-Usaha-Perusahaan-Asuransi,-Asuransi-Syariah,-Reasuransi,-dan-Reasuransi-Syariah/pojk%2069-2016.pdf",
      "doc_id": "1060",
      "page_number": "7",
      "relevance_score": 0.9914557
    },
    "page_content": "Asuransi Syariah dan memenuhi persyaratan untuk \nmewakili Perusahaan Asuransi atau Perusahaan \nAsuransi Syariah memasarkan produk asuransi atau \nproduk asuransi syariah sebagaimana dimaksud \ndalam Undang-Undang Nomor 40 Tahun 2014 

#### **Streaming**

In [ ]:
# from chain.rag_chain import print_answer_stream

# print_answer_stream(chain=chain_history, question="Siapa itu lionel messi?", user_id=USER_ID, conversation_id=CONVERSATION_ID)

Saya minta maaf, tetapi pertanyaan yang Anda ajukan tidak terkait dengan konteks dan informasi metadata yang diberikan. Silakan ajukan pertanyaan lain yang terkait dengan kepatuhan perbankan di Indonesia.